In [37]:
#leyendo texto 1 de prueba
with open('Academic/nf0069.txt') as f:
    contents=f.read()
    print(contents)

nf0069: when talking about the French gangster film very often and 
paradoxically in French one talks about the policier that's the name of the 
genre so when you you see the word policier er I-E police which relates to the 
police it doesn't necessarily mean that it's to do with a police film or police 
procedure as the you have in the American genres but it's co-, covers all er 
virtually all the films er to do with crime er pretty much from the nineteen-
fifties onwards so er a gangster film and a police film will both be called 
policier in French and sometimes you'll see the in slang this becomes polar er 
i'm sorry i hope it's not too awkward if i move er polar is P-O-L-A-R er is yep 
sf0070: sorry 
nf0069: okay we're just beginning polar is the slang word for policier so le 
polar refers to the genre of the police crime gangster movie in French er 
specifically from the nineteen-fifties late forties fifties onwards and so that 
i will 
talk today about a particular genre or subg

In [30]:
##quitar cosas () y []
def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

In [31]:
contents = a(contents)

In [32]:
contents

"nf0069: when talking about the French gangster film very often and \nparadoxically in French one talks about the policier that's the name of the \ngenre so when you you see the word policier er I-E police which relates to the \npolice it doesn't necessarily mean that it's to do with a police film or police \nprocedure as the you have in the American genres but it's co-, covers all er \nvirtually all the films er to do with crime er pretty much from the nineteen-\nfifties onwards so er a gangster film and a police film will both be called \npolicier in French and sometimes you'll see the in slang this becomes polar er \ni'm sorry i hope it's not too awkward if i move er polar is P-O-L-A-R er is yep \nsf0070: sorry \nnf0069: okay we're just beginning polar is the slang word for policier so le \npolar refers to the genre of the police crime gangster movie in French er \nspecifically from the nineteen-fifties late forties fifties onwards and so that \ni will \ntalk today about a particula

In [3]:
#dividir por salto de linea
contents= contents.split('\n')

In [4]:
len(contents)

578

In [5]:
#dividir en oraciones, importar la funcion
from nltk import sent_tokenize

#dividir en oraciones
sentences = []
for sentence in contents:
    sent_token = sent_tokenize(sentence)
    sentences+=[sent_token]

In [6]:
#aplanar la lista de strings
flat_sentences = []
for item in sentences:
        for word in item:
            flat_sentences+=[word]

In [7]:
import re

def remove_punctuation(s):
    return re.sub(r'[^\w\s]','',s)

In [8]:
flat_sentences_nopunc =[]

for sent in flat_sentences:
    flat_sentences_nopunc+=[remove_punctuation(sent)]

In [35]:
flat_sentences_nopunc[:15]

['nf0069 when talking about the French gangster film very often and',
 'paradoxically in French one talks about the policier thats the name of the',
 'genre so when you you see the word policier er IE police which relates to the',
 'police it doesnt necessarily mean that its to do with a police film or police',
 'procedure as the you have in the American genres but its co covers all er',
 'virtually all the films er to do with crime er pretty much from the nineteen',
 'fifties onwards so er a gangster film and a police film will both be called',
 'policier in French and sometimes youll see the in slang this becomes polar er',
 'im sorry i hope its not too awkward if i move er polar is POLAR er is yep',
 'sf0070 sorry',
 'nf0069 okay were just beginning polar is the slang word for policier so le',
 'polar refers to the genre of the police crime gangster movie in French er',
 'specifically from the nineteenfifties late forties fifties onwards and so that',
 'i will',
 'talk today about a